In [1]:
# module
import numpy as np
import pandas as pd
import tqdm
from haversine import haversine
from tqdm import tqdm_pandas, tqdm

In [4]:
#load data
cw = pd.read_csv('횡단보도_완료.csv')
cafe = pd.read_csv('서울강서구_카페현황.csv')

In [6]:
cafe = cafe[['업소명', 'Latitude', 'Longitude', '영업장면적']]

In [10]:
# 차 사고
cw.columns = ['crosswalk_id', 'cw_lon', 'cw_lat']

# 횡단보도
cafe.columns = ['cafe', 'cafe_lat', 'cafe_lon', 'cafe_area']

In [11]:
# key 만들기
cw['key'] = 0
cafe['key'] = 0

C:\Users\abc\anaconda3\envs\please\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# 데이터 합치기
prod = pd.merge(cw, cafe, on = 'key', how = 'outer')

In [13]:
## 하버사인 거리
lat1 = np.deg2rad(prod['cw_lat'])
lat2 = np.deg2rad(prod['cafe_lat'])
lon1 = np.deg2rad(prod['cw_lon'])
lon2 = np.deg2rad(prod['cafe_lon'])
dlat = lat2 - lat1 
dlon = lon2 - lon1 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a))

In [14]:
# 미터거리 구하기
prod['distance'] = c * 6371 * 1000

In [15]:
# 상세한 확인
prod['distance'].describe()

count    2.378508e+06
mean     2.684216e+03
std      1.533816e+03
min      3.948935e+00
25%      1.559579e+03
50%      2.486173e+03
75%      3.546023e+03
max      1.873170e+04
Name: distance, dtype: float64

In [38]:
p100 = prod[prod['distance'] <= 100]
cw_cafe_100 = p100.groupby('crosswalk_id').count().reset_index().iloc[:,:2]
cw_cafe_100.columns = ['crosswalk_id', 'cafe_count']
cw_cafe_100.to_csv('횡단보도_100미터_카페.csv', index = False, encoding = 'cp949')
cw_cafe_100.describe()

,cafe_count
count,1513.000000
mean,3.007931
std,2.671115
min,1.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,20.000000


In [39]:
p50 = prod[prod['distance'] <= 50]
cw_cafe_50 = p50.groupby('crosswalk_id').count().reset_index().iloc[:,:2]
cw_cafe_50.columns = ['crosswalk_id', 'cafe_count']
cw_cafe_50.to_csv('횡단보도_50미터_카페.csv', index = False, encoding = 'cp949')
cw_cafe_50.describe()

,cafe_count
count,736.000000
mean,1.692935
std,1.247912
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,11.000000


In [43]:
stabucks = prod[prod['cafe'].str.contains('스타벅스')]

In [44]:
# 거리가 가까운 순서대로 정렬
p = stabucks.sort_values(by=['distance'])

In [47]:
# 가장 가까운 값들로 저장
pp = p.drop_duplicates(subset = 'crosswalk_id')[['crosswalk_id', 'cafe', 'distance', 'cafe_area']]

In [49]:
pp.to_csv('횡단보도_가장가까운_스타벅스.csv', index = False, encoding = 'cp949')